In [49]:
import os
import torch
import torch.nn as nn
import torchaudio
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
import torchaudio
from torch.utils import data
import editdistance
import soundfile


In [50]:
# class TextTransform:
#     """Maps characters to integers and vice versa"""
#     def __init__(self, char_map_str):
#         self.char_map_str = char_map_str
#         self.char_map = {}
#         self.index_map = {}
#         for line in self.char_map_str.strip().split('\n'):
#             ch, index = line.split()
#             self.char_map[ch] = int(index)
#             self.index_map[int(index)] = ch
#         self.index_map[1] = ' '

#     def text_to_int(self, text):
#         """ Use a character map and convert text to an integer sequence """
#         int_sequence = []
#         for c in text:
#             if c == ' ':
#                 ch = self.char_map['']
#             else:
#                 ch = self.char_map[c]
#             int_sequence.append(ch)
#         return int_sequence

#     def int_to_text(self, labels):
#         """ Use a character map and convert integer labels to a text sequence """
#         string = []
#         for i in labels:
#             string.append(self.index_map[i])
#         return ''.join(string).replace('', ' ')

# # Define the char_map_str
# char_map_str = """
#  ' 0
#  <SPACE> 1
#  a 2
#  b 3
#  c 4
#  d 5
#  e 6
#  f 7
#  g 8
#  h 9
#  i 10
#  j 11
#  k 12
#  l 13
#  m 14
#  n 15
#  o 16
#  p 17
#  q 18
#  r 19
#  s 20
#  t 21
#  u 22
#  v 23
#  w 24
#  x 25
#  y 26
#  z 27
# """

# # Initialize TextTransform with char_map_str
# text_transform = TextTransform(char_map_str)

# # Define train_audio_transforms, valid_audio_transforms, and data_processing as in the original code
# train_audio_transforms = nn.Sequential(
#     torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
#     torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
#     torchaudio.transforms.TimeMasking(time_mask_param=35)
# )

# valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

# def data_processing(data, data_type="train"):
#     spectrograms = []
#     labels = []
#     input_lengths = []
#     label_lengths = []
#     for (waveform, _, utterance, _, _, _) in data:
#         if data_type == 'train':
#             spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
#         else:
#             spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
#         spectrograms.append(spec)
#         label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
#         labels.append(label)
#         input_lengths.append(spec.shape[0] // 2)
#         label_lengths.append(len(label))

#     spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
#     labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

#     return spectrograms, labels, input_lengths, label_lengths

In [51]:
char_map_str = """
' 0
<SPACE> 1
a 2
b 3
c 4
d 5
e 6
f 7
g 8
h 9
i 10
j 11
k 12
l 13
m 14
n 15
o 16
p 17
q 18
r 19
s 20
t 21
u 22
v 23
w 24
x 25
y 26
z 27
"""
 
class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self, char_map_str):
        self.char_map_str = char_map_str
        self.char_map = {}
        self.index_map = {}
        for line in self.char_map_str.strip().split('\n'):
            print("Line:", line)  # Add this line for debugging
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']  # Use '<SPACE>' key for space character
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('', ' ')

text_transform = TextTransform(char_map_str)


train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

# text_transform = TextTransform()


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

Line: ' 0
Line: <SPACE> 1
Line: a 2
Line: b 3
Line: c 4
Line: d 5
Line: e 6
Line: f 7
Line: g 8
Line: h 9
Line: i 10
Line: j 11
Line: k 12
Line: l 13
Line: m 14
Line: n 15
Line: o 16
Line: p 17
Line: q 18
Line: r 19
Line: s 20
Line: t 21
Line: u 22
Line: v 23
Line: w 24
Line: x 25
Line: y 26
Line: z 27


In [52]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 

class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)
        
class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [53]:
criterion = nn.CTCLoss(blank=28).to(torch.device('cuda'))

In [54]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [55]:
# initialize experiment object
experiment = Experiment(api_key="IQcdLgDZdrOWwdXP8KdNHBwjz", project_name="speechrecognition")
experiment.set_name("mlFinal")


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : neat_grade_4875
COMET INFO:     url                   : https://www.comet.com/brody-adams21/speechrecognition/442b49fb6e614c7e8da299371cca9ebf
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     cer                      : 1.0
COMET INFO:     test_loss [3]            : (3.0814789930979414, 3.809693892796835)
COMET INFO:     train_learning_rate [18] : (2.0000000000314905e-09, 0.00047619057142857143)
COMET INFO:     train_loss [18]          : (2.905561685562134, 7.779111385345459)
COMET INFO:     wer                      : 1.0
COMET INFO:   Pa

In [56]:
def levenshtein_distance(ref, hyp):
    """ Compute the Levenshtein distance between two lists """
    m, n = len(ref), len(hyp)
    # Create a distance matrix
    D = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the edge values
    for i in range(m + 1):
        D[i][0] = i
    for j in range(n + 1):
        D[0][j] = j

    # Populate matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                cost = 0
            else:
                cost = 1
            D[i][j] = min(
                D[i - 1][j] + 1,   # Deletion
                D[i][j - 1] + 1,   # Insertion
                D[i - 1][j - 1] + cost  # Substitution
            )

    return D[m][n]

def cer(target, prediction):
    target = target.replace(" ", "")
    prediction = prediction.replace(" ", "")
    distance = levenshtein_distance(target, prediction)
    return distance / max(len(target), len(prediction))


In [57]:
import jiwer


def wer(pred, target):
    """
    Calculate the Word Error Rate (WER).
    
    Returns 0.0 if either input is an empty string, indicating potential silence or
    non-speech segments where WER calculation does not make sense.
    """
    # Ensure inputs are strings and remove leading/trailing whitespace
    pred = str(pred).strip()
    target = str(target).strip()

    # Check for empty strings
    if not pred or not target:
        # Option 1: Return worst case scenario (100% error)
        return 1.0
    
        # Option 2: Return special code or throw an exception to handle it differently
        # raise ValueError("Empty strings provided for WER calculation.")
        # return "Invalid"  # or any indicator to handle these cases distinctly

    # Assuming inputs are now valid and non-empty
    return jiwer.wer(target, pred)



In [58]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment):
    model.train()
    data_len = len(train_loader.dataset)
    with experiment.train():
        for batch_idx, _data in enumerate(train_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            experiment.log_metric('loss', loss.item(), step=iter_meter.get())
            experiment.log_metric('learning_rate', scheduler.get_lr(), step=iter_meter.get())

            optimizer.step()
            scheduler.step()
            iter_meter.step()
            if batch_idx % 100 == 0 or batch_idx == data_len:
                print(r'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion, epoch, iter_meter, experiment):
    print('\nevaluating…')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with experiment.test():
        with torch.no_grad():
            for I, _data in enumerate(test_loader):
                spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                test_loss += loss.item() / len(test_loader)

                decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                    test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)
    experiment.log_metric('test_loss', test_loss, step=iter_meter.get())
    experiment.log_metric('cer', avg_cer, step=iter_meter.get())
    experiment.log_metric('wer', avg_wer, step=iter_meter.get())

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))


def main(
        learning_rate=5e-4, batch_size=20, epochs=10,
        train_url="train-clean-100", test_url="test-clean",
        experiment=Experiment(api_key='', disabled=True)):
    

    hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 29,
        "n_feats": 128,
        "stride": 2,
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

    experiment.log_parameters(hparams)

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")

    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=train_url, download=True)
    test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)
    

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)

    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=28).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')

    iter_meter = IterMeter()
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
        test(model, device, test_loader, criterion, epoch, iter_meter, experiment)

COMET WARNING: Can not parse empty Comet API key
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : mlFinal
COMET INFO:     url                   : https://www.comet.com/brody-adams21/speechrecognition/93a03b0e86a749ab9337ad8a9d26eaff
COMET INFO:   Others:
COMET INFO:     Name : mlFinal
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to ca

In [59]:
main(learning_rate=5e-4, batch_size=40, epochs=3, train_url="train-clean-100", test_url="test-clean", experiment = Experiment(api_key="IQcdLgDZdrOWwdXP8KdNHBwjz", project_name="speechrecognition"))

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in 'c:\\Users\\brody\\Machine Learning\\Final Project' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/brody-adams21/speechrecognition/c3741943eb9e4e0b8e7a55b471c5f950



SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

In [60]:
experiment.end()
 